In [ ]:
! pip install -U spacy -q 

In [ ]:
!python -m spacy info

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-04-01 01:49:41.212945: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-01 01:49:43.172961: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

============================== Info about spaCy ==============================

spaCy version    3.5.1                         
Location         /usr/local/lib/python3.9/dist-packages/spacy
Platform         Linux-5.10.147+-x86_64-with-glibc2.31
Python version   3.9.16                        
Pipelines        en_core_web_sm (3.5.0)        



In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [ ]:
import json
f = open('/content/crop_annotation.json')
TRAIN_DATA = json.load(f)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 20/20 [00:00<00:00, 1039.58it/s]

Skipping entity
Skipping entity
Skipping entity


In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-04-01 01:49:49.132132: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-01 01:49:51.598888: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-04-01 01:49:58.665063: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-01 01:50:00.482695: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-04-01 01:50:01,549] [INFO] Set up nlp object from config
[2023-04-01 01:50:01,569] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-01 01:50:01,577] [INFO] Created vocabulary
[2023-04-01 01:50:01,578] [INFO] Finished initializing nlp object
[2023-04-01 01:50:01,774] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pip

In [ ]:
nlp_ner = spacy.load("/content/model-best") 

In [ ]:
doc = nlp_ner('''The proportion of the crop cycle that takes place in one year The average actual annual yield taking into account the Energy or number of chews necessary to chew the banana to make it ready to be swallowed Aroma of fresh grass Elemental flavor caused by dilute aqueous solutions of various substances such as sucrose or aspartame Uniformity of color of the surface of the sample''')

In [ ]:
from spacy import displacy
spacy.displacy.render(doc, style="ent", jupyter=True) 

In [ ]:
doc = nlp_ner('The visual aspect of boiled potatoes: potatoes turn gray or dark after boiling')
spacy.displacy.render(doc, style="ent", jupyter=True) 

In [ ]:
doc = nlp_ner('The tolerance to drought of the plant')
spacy.displacy.render(doc, style="ent", jupyter=True) 

In [ ]:
doc = nlp_ner('Cooked carrot amaranth and beet root sweetpotato, cassava, and balugu climbing yam Gustatory sensation that remains The content of carbohydrate in the seed.')
spacy.displacy.render(doc, style="ent", jupyter=True)

## Probability of ner predicted by model

In [ ]:
import spacy
from collections import defaultdict

texts = ['The proportion of the crop cycle that takes place in one year The average actual annual yield taking into account the Energy or number of chews necessary to chew the banana to make it ready to be swallowed Aroma of fresh grass Elemental flavor caused by dilute aqueous solutions of various substances such as sucrose or aspartame Uniformity of color of the surface of the sample']

beam_width = 16
beam_density = 0.0001 
nlp_ner = spacy.load("/content/model-best") 


docs = list(nlp.pipe(texts, disable=['ner']))
beams = nlp.get_pipe("ner").beam_parse(docs, beam_width=beam_width, beam_density=beam_density)

for doc, beam in zip(docs, beams):
    entity_scores = defaultdict(float)
    for score, ents in nlp.get_pipe("ner").moves.get_beam_parses(beam):
        for start, end, label in ents:
            entity_scores[(start, end, label)] += score

l= []
for k, v in entity_scores.items():
    l.append({'start': k[0], 'end': k[1], 'label': k[2], 'prob' : v} )

for a in sorted(l, key= lambda x: x['start']):
    print(a)
    
'''
Based on model probability score it can be easily predicted that model is overfitted beacuse of less training data 

'''


{'start': 10, 'end': 12, 'label': 'AVERAGE ANNUAL CYCLE', 'prob': 1.0}
{'start': 21, 'end': 26, 'label': 'QUALITY', 'prob': 1.0}
{'start': 38, 'end': 42, 'label': 'AROMA', 'prob': 1.0}
{'start': 54, 'end': 57, 'label': 'TASTE', 'prob': 1.0}
{'start': 57, 'end': 60, 'label': 'APPEARENCE', 'prob': 1.0}
